In [6]:
from Database import DatabaseInputs

In [7]:
db_input = DatabaseInputs()
input = db_input.get_input()

In [8]:
input

{'RISK': '0.5',
 'CURRENCY': 'GBP',
 'LOGIN': '51185390',
 'PASSWORD': 'rpEqG5ny',
 'SERVER': 'ICMarketsSC-Demo',
 'FOLDER_PATH': 'C:/Users/pjesus/Documents/TradingView'}

In [4]:
input['FOLDER_PATH']

'C:\\Users\\pjesus\\Documents\\TradingView'

In [11]:
from tinydb import TinyDB

# create a TinyDB object and open the database file
db = TinyDB('db/my_database.json')

# create two tables in the database
inputs = db.table('INPUT')

# add some data to table1
inputs.insert({
    "USE_CAPITAL_LIQUID_MT5" : "SIM",
    "FIXED_CAPITAL": "300",
    "RISK": "0.5", 
    "CURRENCY": "GBP", 
    "LOGIN": "51185390", 
    "PASSWORD": "rpEqG5ny", 
    "SERVER": "ICMarketsSC-Demo", 
    "FOLDER_PATH": "C:/Users/pjesus/Documents/TradingView"
})

# close the database connection
db.close()


In [7]:
from tinydb import TinyDB

# Open the database
db = TinyDB('my_database.json')

# Get a reference to the table
table = db.table('INPUT')

# Select all data from the table
results = table.all()

# Print the results
print(results[0])

# Close the database
db.close()


0.5


In [ ]:
# Get a reference to the table
table = db.table('mytable')

# Select all data from the table
results = table.all()

# Print the results
print(results)

In [2]:
from tinydb import TinyDB, Query

# Open the database
db = TinyDB('my_database.json')

# Get a reference to the 'users' table
inputs_table = db.table('inputs')

# Define a query to find all users with the name 'Alice'
User = Query()
results = inputs_table.search(User.login == '51185390')

# Print the results
print(results)

# Close the database
db.close()


[{'risk': '0.5', 'currency': 'GBP', 'login': '51185390', 'password': 'rpEqG5ny', 'server': 'ICMarketsSC-Demo', 'folder_path': 'C:\\Users\\pjesus\\Documents\\TradingView'}]


In [24]:
a = 0.00223

In [25]:
int(round(a*100,0))

0

In [6]:
b = float(0.00)

In [7]:
b

0.0

In [2]:
a

0.00123

In [5]:
a > 0.00

True

In [13]:
import datetime

# Get the current timestamp as a string
current_timestamp_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

print(current_timestamp_str)


2023-05-06 15:36:53


In [33]:
'''https://www.mql5.com/en/docs/python_metatrader5'''

import os
import math
import pandas as pd
import MetaTrader5 as mt5
from datetime import datetime

from Order import Order


In [5]:

risk = 0.5
currency = 'gbp'
currency = currency.upper()
deviation = 10
login = 51185390
password = 'rpEqG5ny'
server = 'ICMarketsSC-Demo'
folder_path = r"C:\Users\pjesus\Documents\TradingView"


In [38]:
my_orders_list

[['EURUSD', 'Buy', 'Limit', 1.09386, 1.0986, 1.0932],
 ['USDCAD', 'Buy', 'Limit', 1.3349, 1.34724, 1.334],
 ['USDJPY', 'Buy', 'Limit', 133.787, 134.553, 133.666],
 ['EURUSD', 'Buy', 'Limit', 1.09223, 1.0986, 1.09157],
 ['EURUSD', 'Buy', 'Limit', 1.09137, 1.0986, 1.09096],
 ['USDCAD', 'Buy', 'Limit', 1.3321, 1.34737, 1.33124],
 ['USDJPY', 'Buy', 'Limit', 134.047, 134.551, 133.963],
 ['USDJPY', 'Buy', 'Limit', 134.288, 134.792, 134.193]]

In [41]:
my_list = [[3, 'apple'], [1, 'banana'], [2, 'cherry']]
a = my_list.sort(key=lambda x: x[0])
a

In [43]:
print(a)

None


In [6]:
# pega o arquivo exportado mais atual

folder_path = r"C:\Users\pjesus\Documents\TradingView"

file_list = []

for file_name in os.listdir(folder_path):
    if 'paper-trading-orders' in file_name:
        full_path = os.path.join(folder_path, file_name)
        if os.path.isfile(full_path):
            mod_time = os.stat(full_path).st_mtime
            file_list.append((full_path, file_name, mod_time))

try:
    sorted_files = sorted(file_list, key=lambda x: x[2], reverse=True)
    sorted_files = [[file[0], file[1], datetime.fromtimestamp(file[2]).strftime('%Y-%m-%d %H:%M:%S')] for file in sorted_files]
    last_file = sorted_files[0][0]


except Exception as exp:
    new_log(f'{exp}')


# le o arquivo exportado para criar as ordens

orders_list = []



# Read CSV file and use first row as column headers
orders_df = pd.read_csv(last_file, header=0)

# Loop through each row of the DataFrame
for index, row in orders_df.iterrows():
    # Access data from each column using the column name as a key

    try: # file of all orders
        if row['Type'] == 'Limit' and row['Status'] == 'Working':

            orders_list.append([
                row['Symbol'], 
                row['Side'], 
                row['Type'],
                row['Price'],
                row['Take Profit'],
                row['Stop Loss']
            ])
    except:
        try: # file of working orders
            if row['Type'] == 'Limit':

                orders_list.append([
                    row['Symbol'], 
                    row['Side'], 
                    row['Type'],
                    row['Price'],
                    row['Take Profit'],
                    row['Stop Loss']
                ])
        except Exception as excp:
            print(str(excp))


# remove order duplicadas no arquivo

my_set = set(tuple(x) for x in orders_list)
my_orders_list = [list(x) for x in my_set]

orders_list = []


# cria os objetos de ordens
for symbol, side, type, entry, take_profit, stop_loss in my_orders_list:

    symbol = symbol.split(":")[1]
    symbol_ref = currency + symbol[3:]

    if currency == symbol[3:]:
        symbol_ref = symbol

    try:

        order = Order(
            symbol      = symbol, 
            symbol_ref  = symbol_ref,
            side        = side, 
            type        = type,
            entry       = float(entry), 
            take_profit = float(take_profit), 
            stop_loss   = float(stop_loss)        
        )

        orders_list.append(order)

    except Exception as exp:

        print(symbol, side, type, entry, take_profit, stop_loss)
        print(str(exp))


In [7]:
for order in orders_list:

    print(order.symbol, order.side, order.entry, order.take_profit, order.stop_loss, order.symbol_ref)

USDJPY Buy 134.047 134.551 133.963 GBPJPY
EURUSD Buy 1.09223 1.0986 1.09157 GBPUSD
EURUSD Buy 1.09137 1.0986 1.09096 GBPUSD
USDCAD Buy 1.3349 1.34724 1.334 GBPCAD
USDCAD Buy 1.3321 1.34737 1.33124 GBPCAD
USDJPY Buy 134.288 134.792 134.193 GBPJPY
USDJPY Buy 133.787 134.553 133.666 GBPJPY
EURUSD Buy 1.09386 1.0986 1.0932 GBPUSD


In [5]:
# remove as ordens existentes no metatrader

# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    new_log(f"initialize() failed, error code = {mt5.last_error()}")



authorized = mt5.login(login,password,server)

if authorized:
    # display trading account data 'as is'
    new_log(mt5.account_info())
else:
    new_log(f"failed to connect at account #{login}, error code: {mt5.last_error()}")

# verifica se já existe a ordem posicionada 
orders=mt5.orders_get()

order_to_remove = []

# pega as ordens que ja estão posicionadas e remove
for order in orders:
    order_to_remove.append(order.ticket)

for order_rm in order_to_remove:

    request = {
        "action"       : mt5.TRADE_ACTION_REMOVE,
        "order"        : int(order_rm)
    }

    # send a trading request
    result = mt5.order_send(request)

    # check the execution result
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        new_log("order_remove failed, retcode={}".format(result.retcode))
            
    else:
        new_log(f"order_remove done {result}")
        new_log("removed position with POSITION_TICKET={}".format(result.order))


shutdown = mt5.shutdown()


In [7]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    new_log(f"initialize() failed, error code = {mt5.last_error()}")


authorized = mt5.login(login,password,server)

if authorized:
    new_log(mt5.account_info())
else:
    new_log(f"failed to connect at account #{login}, error code: {mt5.last_error()}")
    
all_symbols = mt5.symbols_get()
all_symbols = [s.name for s in all_symbols]

account_info = mt5.account_info()

balance = account_info.balance

for order in orders_list:

    symbol = order.symbol
    symbol_ref = order.symbol_ref

    symbol_info = mt5.symbol_info(symbol)  # informacoes do ativo principal
    symbol_ref_info = mt5.symbol_info(symbol_ref) # informacoes do ativo referencia para calculo

    point               = symbol_info.point
    digits              = symbol_info.digits
    ask_ref             = round(symbol_ref_info.ask, symbol_ref_info.digits) # referencia ao par da moeda margem
    trade_contract_size = symbol_info.trade_contract_size

    side        = order.side
    entry       = round(order.entry,digits - 1)
    take_profit = round(order.take_profit,digits - 1)
    stop_loss   = round(order.stop_loss,digits - 1)

    pip_value     = round((point*10/ask_ref)*trade_contract_size,2)
    pip_stop_loss = round(round(abs(order.entry - order.stop_loss),5)*math.pow(10,digits - 1),1)
    lot_size      = round((balance * (risk/100))/(pip_stop_loss*pip_value),2)

    type_order = mt5.ORDER_TYPE_BUY_LIMIT if side == 'Buy' else mt5.ORDER_TYPE_SELL_LIMIT

    request = {
        "action"       : mt5.TRADE_ACTION_PENDING,
        "symbol"       : symbol,
        "volume"       : lot_size,
        "type"         : type_order,
        "price"        : entry,
        "sl"           : stop_loss,
        "stoplimit"    : stop_loss,
        "tp"           : take_profit,
        "deviation"    : deviation,                             
        "magic"        : 0,
        "comment"      : f"{pip_stop_loss} pips",
        "type_time"    : mt5.ORDER_TIME_GTC,
        "type_filling" : mt5.ORDER_FILLING_RETURN,
    }


    # send a trading request
    result = mt5.order_send(request)

    # check the execution result
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        new_log("order_send failed, retcode={}".format(result.retcode))
    else:
        new_log(f"order_send done {result} ")
        new_log("opened position with POSITION_TICKET={}".format(result.order))


    new_log(f''' 
    _________________________________________________________________________________________________
    | {side}                                                                                     
    | Instrument         : {symbol}     Deposit currency : {currency}                             
    | Open price         : {entry}      Stop loss price  : {stop_loss}  ({pip_stop_loss} pips)    
    | Account Balance    : {balance}    Risk (%)         : {risk}%                                
    | {symbol_ref} price : {ask_ref}    Contract size (Units per Lot) : {int(trade_contract_size)}

    | Lots > {lot_size}
    | Money at risk > {(300 * (risk/100))}
    _________________________________________________________________________________________________

    ''') 



shutdown = mt5.shutdown()

In [16]:
a = 0.0012324
b = 0.01012312

In [21]:
aa = round(a,2)
aa

0.0

In [22]:
bb = round(b,2)
bb

0.01

In [24]:
aa > 0.00

False

In [25]:
bb > 0.00

True

In [13]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    new_log(f"initialize() failed, error code = {mt5.last_error()}")


authorized = mt5.login(login,password,server)

if authorized:
    # display trading account data 'as is'
    new_log(mt5.account_info())
else:
    new_log(f"failed to connect at account #{login}, error code: {mt5.last_error()}")
    

all_symbols = mt5.symbols_get()
all_symbols = [s.name for s in all_symbols]

account_info = mt5.account_info()

balance = account_info.balance
balance = 300

for order in orders_list:

    symbol = order.symbol
    symbol_ref = order.symbol_ref

    symbol_info = mt5.symbol_info(symbol)  # informacoes do ativo principal
    symbol_ref_info = mt5.symbol_info(symbol_ref) # informacoes do ativo referencia para calculo

    point               = symbol_info.point
    digits              = symbol_info.digits
    ask_ref             = round(symbol_ref_info.ask, symbol_ref_info.digits) # referencia ao par da moeda margem
    trade_contract_size = symbol_info.trade_contract_size

    side        = order.side
    entry       = round(order.entry,digits - 1)
    take_profit = round(order.take_profit,digits - 1)
    stop_loss   = round(order.stop_loss,digits - 1)

    pip_value     = round((point*10/ask_ref)*trade_contract_size,2)
    pip_stop_loss = round(round(abs(order.entry - order.stop_loss),5)*math.pow(10,digits - 1),0)
    lot_size      = round((balance * (risk/100))/(pip_stop_loss*pip_value),2)


    print(f''' 
    _________________________________________________________________________________________________
    | {side}                                                                                     
    | Instrument         : {symbol}     Deposit currency : {currency}                             
    | Open price         : {entry}      Stop loss price  : {stop_loss}  ({pip_stop_loss} pips)    
    | Account Balance    : {balance}    Risk (%)         : {risk}%                                
    | {symbol_ref} price : {ask_ref}    Contract size (Units per Lot) : {int(trade_contract_size)}
    |
    | Lots -> {lot_size} +++ {(balance * (risk/100))/(pip_stop_loss*pip_value)}
    | Money at risk -> {(300 * (risk/100))}
    _________________________________________________________________________________________________
    ''') 



shutdown = mt5.shutdown()

 
    _________________________________________________________________________________________________
    | Buy                                                                                     
    | Instrument         : USDJPY     Deposit currency : GBP                             
    | Open price         : 134.05      Stop loss price  : 133.96  (8.0 pips)    
    | Account Balance    : 300    Risk (%)         : 0.5%                                
    | GBPJPY price : 170.312    Contract size (Units per Lot) : 100000
    |
    | Lots -> 0.03 +++ 0.031942078364565585
    | Money at risk -> 1.5
    _________________________________________________________________________________________________
    
 
    _________________________________________________________________________________________________
    | Buy                                                                                     
    | Instrument         : EURUSD     Deposit currency : GBP                          

In [29]:
class Person:
    def __init__(self, name, age):
        self.name = name
        self.age = age

people = [Person('Alice', 25), Person('Bob', 30), Person('Charlie', 20)]

people.sort(key=lambda x: x.age)

In [30]:
for p in people:

    print(p.name)

Charlie
Alice
Bob


In [17]:
import math

x = 0.187056277056277056
rounded_down = math.floor(x)
print(rounded_down)  # output: 0.02


0


In [29]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    new_log(f"initialize() failed, error code = {mt5.last_error()}")


authorized = mt5.login(login,password,server)

if authorized:
    # display trading account data 'as is'
    new_log(mt5.account_info())
else:
    new_log(f"failed to connect at account #{login}, error code: {mt5.last_error()}")

all_symbols = mt5.symbols_get()
all_symbols = [s.name for s in all_symbols]

print(all_symbols)

shutdown = mt5.shutdown()

['EURUSD', 'GBPUSD', 'USDCHF', 'USDJPY', 'USDCAD', 'AUDUSD', 'AUDNZD', 'AUDCAD', 'AUDCHF', 'AUDJPY', 'CHFJPY', 'EURGBP', 'EURAUD', 'EURJPY', 'EURCHF', 'EURNZD', 'EURCAD', 'GBPCHF', 'GBPJPY', 'CADCHF', 'CADJPY', 'GBPAUD', 'GBPCAD', 'GBPNZD', 'NZDCAD', 'NZDCHF', 'NZDJPY', 'NZDUSD', 'USDSGD', 'AUDSGD', 'CHFSGD', 'EURDKK', 'EURHKD', 'EURNOK', 'EURPLN', 'EURSEK', 'EURSGD', 'EURTRY', 'EURZAR', 'GBPDKK', 'GBPNOK', 'GBPSEK', 'GBPSGD', 'GBPTRY', 'NOKJPY', 'NOKSEK', 'SEKJPY', 'SGDJPY', 'USDCNH', 'USDCZK', 'USDDKK', 'USDHKD', 'USDHUF', 'USDMXN', 'USDNOK', 'USDPLN', 'USDRUB', 'USDSEK', 'USDTHB', 'USDTRY', 'USDZAR', 'AUS200', 'CHINA50', 'ES35', 'STOXX50', 'F40', 'HK50', 'IT40', 'JP225', 'UK100', 'US2000', 'US30', 'US500', 'USTEC', 'XAGEUR', 'XAGUSD', 'XAUEUR', 'XAUUSD', 'XPDUSD', 'XPTUSD', 'XBRUSD', 'XNGUSD', 'XTIUSD', 'BTCUSD', 'BCHUSD', 'ETHUSD', 'LTCUSD', 'DSHUSD', 'AAPL.NAS', 'ADBE.NAS', 'ADI.NAS', 'ADP.NAS', 'AMAT.NAS', 'AMGN.NAS', 'AMZN.NAS', 'ATVI.NAS', 'AVGO.NAS', 'BKNG.NAS', 'BIIB.NAS', 'C

In [31]:
[s for s in all_symbols if s.startswith(currency) and s.endswith('CHF')]

['USDCHF']